In [ ]:
import os
import gc
import cv2
import time
import random

# For data manipulation
import numpy as np
import pandas as pd

# Pytorch Imports
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.optim import lr_scheduler
from torch.utils.data import Dataset, DataLoader
from torch.cuda import amp
from pytorch_toolbelt import losses as L

# Utils
from tqdm.auto import tqdm

# For Image Models
import timm

# Albumentations for augmentations
import albumentations as A
from albumentations.pytorch import ToTensorV2

## using gpu:1
os.environ['CUDA_VISIBLE_DEVICES'] = '0'

def seed_everything(seed=123):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
seed_everything()

In [ ]:
class Customize_Model(nn.Module):
    def __init__(self, model_name, cls):
        super().__init__()
        self.model = timm.create_model(model_name, 
                                       pretrained=True, 
                                       num_classes=cls, 
                                       drop_rate= CFG['drop_out'], 
                                       drop_path_rate= CFG['drop_path'])
        
    def forward(self, image):
        x = self.model(image)
        return x

In [ ]:
def get_train_transform(img_size):
    return A.Compose([
        A.SmallestMaxSize(max_size=img_size, interpolation=3, p=1),
#         A.Resize(img_size, img_size),
        
        A.RandomBrightnessContrast(brightness_limit=0.2, contrast_limit=0.2, p=0.5),
#         A.HorizontalFlip(p=0.5),
#         A.VerticalFlip(p=0.5),
#         A.Blur(blur_limit= 3, p=0.3), 
        A.GaussNoise(p=0.3),
        A.OneOf([
                A.Cutout(max_h_size=10, max_w_size=16),
                A.CoarseDropout(max_holes=4),
            ], p=0.5),
        A.ShiftScaleRotate(shift_limit=0.05, scale_limit=0.15, rotate_limit= 0,
                                        interpolation=cv2.INTER_LINEAR, border_mode=0, p=0.7),
        ToTensorV2(p=1.0),
    ])


def get_test_transform(img_size):
    return A.Compose([
        A.SmallestMaxSize(max_size=img_size, interpolation=3, p=1),
#         A.Resize(img_size, img_size),
        ToTensorV2(p=1.0),
    ])

In [ ]:
from toolbox.audio_aug import *

class Customize_Dataset(Dataset):
    def __init__(self, df, transforms=None, mixup=0):
        self.df = df
        self.transforms = transforms
        self.mixup= mixup
        
    def mixup_aug(self, img_1, mask_1, 
                        img_2, mask_2):
        """
        img: numpy array of shape (height, width,channel)
        mask: numpy array of shape (height, width,channel)
        """
        ## mixup
        weight= np.random.beta(a=0.4, b=0.4)
        img= img_1*weight + img_2*(1-weight)
        mask= mask_1*weight + mask_2*(1-weight)
        return img.astype(np.uint8), mask
    
    def read_data(self, data):
        img = cv2.imread(data['image_path'])
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        
        label= [0]*CFG['num_classes']
        cls = data['label']
        label[cls]= 1
        label= np.array(label)
        return img, label
    
    def __getitem__(self, index):
        data = self.df.loc[index]
        img, label= self.read_data(data)
        
        # use mixup
        if self.mixup and np.random.rand() >= (1-self.mixup):
            img_1= img
            label_1= np.array(label)
            while True:
                indx= np.random.randint(len(self.df))
                data= self.df.loc[indx]
                img_2, label_2= self.read_data(data)
                if label_1.argmax(0)!=label_2.argmax(0): break
            img, label= self.mixup_aug(img_1, label_1, 
                                       img_2, label_2)
        
        if self.transforms:
            img = self.transforms(image=img)["image"]
            
        return {
            'image': torch.tensor(img/255, dtype=torch.float32),
            'label': torch.tensor(label, dtype=torch.float32),
        }
    
    def __len__(self):
        return len(self.df)

In [ ]:
class Customize_loss(nn.Module):
    def  __init__(self):
        super().__init__()
        self.CrossEntropy= nn.CrossEntropyLoss(weight= cls_weight, label_smoothing=0.25)
        self.FocalCosineLoss= L.FocalCosineLoss()
        self.soft_ce= L.SoftCrossEntropyLoss(smooth_factor=0.25)
        self.bi_temp= L.BiTemperedLogisticLoss(t1=0.8, t2=1.2)
    
    def forward(self, y_pred, y_true):
        loss= 1.0 * self.CrossEntropy(y_pred, y_true)
        return loss

In [ ]:
def train_epoch(dataloader, model, criterion, optimizer):
    scaler= amp.GradScaler()
    model.train()

    ep_loss= []
    for i, data in enumerate(tqdm(dataloader)):

        imgs= data['image'].to('cuda')
        labels= data['label'].to('cuda')
        
        with amp.autocast():
            preds= model(imgs)
            loss= criterion(preds, labels)
            ep_loss.append(loss.item())
            loss/= CFG['gradient_accumulation']
            scaler.scale(loss).backward()
            
            if (i+1) % CFG['gradient_accumulation']== 0:
                scaler.step(optimizer)
                scaler.update()
                optimizer.zero_grad()
                
    return np.mean(ep_loss)

In [ ]:
from metrics import *

def valid_epoch(dataloader, model, criterion):
    model.eval()
    
    ep_loss= []
    all_pred= []
    all_label= []
    for i, data in enumerate(tqdm(dataloader)):

        imgs= data['image'].to('cuda')
        labels= data['label'].to('cuda')
        all_label.extend(labels.cpu().numpy())
        
        with torch.no_grad():
            preds= model(imgs)
            loss= criterion(preds, labels)
            ep_loss.append(loss.item())
        all_pred.extend(preds.cpu().softmax(dim=-1).numpy())
        
    
    ## caculate metrics
    all_label= np.array(all_label).argmax(1)
    all_pred= np.array(all_pred)
    
    acc= Accuracy(all_pred, all_label)
    print(f'accuracy: {acc}')
    recall= Mean_Recall(all_pred, all_label)
    print(f'mean_recall: {recall}')
    
    cmap= padded_cmap(all_pred, all_label)
    print(f'cmap: {cmap}')
    
    score= cmap
    return np.mean(ep_loss), score

# CFG

In [ ]:
timm.list_models(pretrained=True)

In [ ]:
CFG= {
    'fold': 0,
    'epoch': 30,
    'model_name': 'tf_efficientnet_b0_ns',
    'soft_target': 0.9,
    'finetune': True,
    
    'img_size': 128,
    'batch_size': 64,
    'gradient_accumulation': 1,
    'gradient_checkpoint': False,
    'drop_out': 0.3,
    'drop_path': 0.2,
    'mixup': 0.65,
    
    'lr': 6e-5,
    'weight_decay': 5e-4,
    
    'num_classes': 264,
    'load_model': 'pretrained/nfnet_l0.pth',
    'save_model': './train_model'
}

if CFG['finetune']:
    CFG['lr']= 6e-6
    CFG['load_model']= f"./train_model/cv{CFG['fold']}_best.pth"
#     CFG['load_model']= f"./train_model/cv{CFG['fold']}_ep30.pth"
CFG

# Prepare Dataset

In [ ]:
df= pd.read_csv('Data/train.csv')
label_name= list(df['label_name'].unique())

df_1= pd.read_csv('Data/train_ex2020.csv')
df_2= pd.read_csv('Data/train_ex2021.csv')
df_3= pd.read_csv('Data/train_ex2022.csv')
df_4= pd.read_csv('Data/train_ex1.csv')
ex_df= pd.concat([df_1, df_2, df_3, df_4], axis=0)
ex_df= ex_df[ex_df['label_name'].isin(label_name)].reset_index(drop=True)

la= list(ex_df['label_name'].unique())
for l in la:
    ex_df.loc[ ex_df['label_name']==l, 'label' ]= label_name.index(l)
ex_df

In [ ]:
df= pd.read_csv('Data/train.csv')

train_dataset= df[df['fold']!=CFG['fold']].reset_index(drop=True)
train_dataset= pd.concat([train_dataset, ex_df],axis=0).reset_index(drop=True)
label= train_dataset['label'].unique().tolist()

from sklearn.utils.class_weight import compute_class_weight
cls_weight= compute_class_weight(class_weight='balanced', classes=list(range(264)), y=train_dataset['label'].values)
cls_weight= torch.tensor(cls_weight).cuda()

valid_dataset= df[df['fold']==CFG['fold']].reset_index(drop=True)
print(f'train dataset: {len(train_dataset)}')
print(f'valid dataset: {len(valid_dataset)}')

train_dataset= Customize_Dataset(train_dataset, get_train_transform(CFG['img_size']), mixup=CFG['mixup'])
valid_dataset= Customize_Dataset(valid_dataset, get_test_transform(CFG['img_size']), mixup=False)

train_loader= DataLoader(train_dataset, batch_size= CFG['batch_size'], shuffle=True, num_workers=0)
valid_loader= DataLoader(valid_dataset, batch_size=32, shuffle=False, num_workers=0)
df.head()

# Train

In [ ]:
## create model
if CFG['load_model']:
    print(f"load_model: {CFG['load_model']}")
    model= torch.load(CFG['load_model'], map_location= 'cuda')
else:
    model= Customize_Model(CFG['model_name'], CFG['num_classes'])
    
if CFG['gradient_checkpoint']: 
    print('use gradient checkpoint')
    model.model.set_grad_checkpointing(enable=True)
    
if 'pretrained' in str(CFG['load_model']): 
    print('use bird pretrained')
#     model.model.classifier= nn.Linear(in_features=1280, out_features=264, bias=True)
    model.model.head.fc= nn.Linear(in_features=2304, out_features=264, bias=True)
model.to('cuda')
    
## hyperparameter
criterion= Customize_loss()
optimizer= optim.AdamW(model.parameters(), lr= CFG['lr'], weight_decay= CFG['weight_decay'])

## start training
best_score= 0
for ep in range(1, CFG['epoch']+1):
    print(f'\nep: {ep}')
    
    train_loss= train_epoch(train_loader, model, criterion, optimizer)
    valid_loss, valid_acc= valid_epoch(valid_loader, model, criterion)
    print(f'train loss: {round(train_loss, 5)}')
    print(f'valid loss: {round(valid_loss, 5)}, valid_acc: {round(valid_acc, 5)}')
    
    if valid_acc >= best_score:
        best_score= valid_acc
        torch.save(model, f"{CFG['save_model']}/cv{CFG['fold']}_best.pth")
        print(f'model save at score: {round(best_score, 5)}')
        
    ## save model every epoch
    torch.save(model, f"{CFG['save_model']}/cv{CFG['fold']}_ep{ep}.pth")